# Sequential Informed Federated Unlearning: Efficient and Provable Client Unlearning in Federated Optimization

The main contribution of this work consists in Informed Federated Unlearning (IFU), a novel efficient FU approach to unlearn a client's contribution with quantifiable unlearning guarantees. IFU requires minimal additional computations to the server in a standard FedAvg procedure. Specifically, the server quantifies at every optimization round each client's contribution to the global model.
Upon receiving an unlearning request from a client, the server identifies in the FL training history the optimal FL iteration and associated intermediate global model from which re-initializing the unlearning procedure. Unlearning guarantees are provided by introducing a novel randomized mechanism to perturb the selected intermediate model with client-specific noise. 
We also extend IFU to Sequential Informed Federated Unlearning (SIFU), to account for realistic unlearning scenarios where the server receives sequential unlearning requests from one or more clients at different times.

## Background

Let us consider a dataset $\mathcal{D}$ composed of two disjoint datasets: $\mathcal{D}_f$, the cohort of data samples on which unlearning must be applied after FL training, and $\mathcal{D}_k$, the remaining data samples. Hence, we have $\mathcal{D} = \mathcal{D}_f \cup \mathcal{D}_k$. 
We also consider $\mathcal{M}(\mathcal{D})$, the ML model parameters resulting from training with optimization scheme $\mathcal{M}$ on dataset $\mathcal{D}$. 
We introduce in this section the different unlearning baselines and methods currently used to unlearn $\mathcal{D}_f$ from the trained model $\mathcal{M}(\mathcal{D})$. 

- **MU through retraining.** Within this setting, a new training is performed from scratch with only $\mathcal{D}_k$ as training data. As the initial model contains no information from $\mathcal{D}_f$, the new trained model $\mathcal{M}(\mathcal{D}_k)$ also contains no information from $\mathcal{D}_f$. We note however that this procedure wastes the contribution of $\mathcal{D}_k$ already available by training originally on $\mathcal{D}$. Hence, this method is considered sub-optimal, and represents a basic baseline for unlearning approaches.

- **MU through fine-tuning.** Fine-tuning on the remaining data $\mathcal{D}_k$ has been proposed as a practical approach to unlearn the specificities of $\mathcal{D}_f$. However, fine-tuning does not provide guarantees about the effectiveness of the unlearning. 

- **MU through model scrubbing.** Another unlearning approach consists in applying a “scrubbing" transformation $h$ to the model $\mathcal{M}(\mathcal{D})$ such that the resulting model is as close as possible to the one that would be trained with only $\mathcal{D}_k$, i.e. $h(\mathcal{M}(\mathcal{D})) \approx \mathcal{M}(\mathcal{D}_k)$. To define a scrubbing method $h$, existing work mostly relies on the following Assumption 1, which considers a quadratic approximation of the loss function.

- **Assumption 1.** For model parameters $\mathbf{\theta}$ and $\mathbf{\phi}$, the gradient of the loss function of a given data point $\mathcal{D}_x$ satisfies
\begin{equation}
\nabla f_{\mathcal{D}_x}(\mathbf{\phi})
= \nabla f_{\mathcal{D}_x}(\mathbf{\theta})
+ H_{\mathcal{D}_x}(\mathbf{\theta})(\mathbf{\phi} - \mathbf{\theta})
,
\end{equation}
where $H_{\mathcal{D}_x}(\mathbf{\theta})$ is positive semi-definite.

The scrubbed model is the new optimum obtained when unlearning data samples in $\mathcal{D}_f$. Hence, under Assumption \ref{ass:linear_approx}, the new optimum can be obtained by setting $\nabla f_{\mathcal{D}_k}(h_{\mathcal{D}_k}(\mathbf{\theta})) = 0 $, which gives
\begin{equation}
h_{\mathcal{D}_k}(\mathbf{\theta})
= \mathbf{\theta} - H_{\mathcal{D}_k}^{-1}(\mathbf{\theta}) \nabla f_{\mathcal{D}_k}(\mathbf{\theta}).
\end{equation}

With the above equation, $h$ reduces to performing a Newton step, and has been derived in previous MU works under different theoretical assumptions that can be generalized with Assumption 1. The main drawback behind the use of the scrubbing function is the computation of the Hessian, which can be unfeasible for high dimensional model. Finally, the scrubbing function is often coupled with Gaussian noise perturbation on the resulting weights, to compensate the quadratic approximation of the loss function or the approximation of the Hessian.

- **MU through noise perturbation.** This unlearning method consists in randomly perturbing the trained model $\mathcal{M}(\mathcal{D})$ to unlearn specificities from data samples in $\mathcal{D}_f$. 
The noise is set such that the guarantees of Definition 1 are satisfied, where $(\epsilon, \delta)$ are parameters quantifying the unlearning guarantees. 

- **Definition 1.** Let $f_m$ be a randomized mechanism taking model parameters as an input. $(\epsilon, \delta)$-Unlearning  trough $f_m$ of a data point $\{x_m, y_m\}$ from a model $\mathcal{M}(D)$  is achieved if, for any subset $\mathcal{S}$ of the model parameters space and $D_{-m} := D \setminus \{x_m, y_m\}$, we have
$$\mathbb{P}(f_m(\mathcal{M}(D)) \in \mathcal{S}) 
\le e^\epsilon \mathbb{P}(f_m(\mathcal{M}(D_{-m})) \in \mathcal{S})  + \delta$$
$$\text{and }
\mathbb{P}(f_m(\mathcal{M}(D_{-m})) \in \mathcal{S}) 
\le e^\epsilon \mathbb{P}(f_m(\mathcal{M}(D)) \in \mathcal{S}) + \delta.$$

## Unlearning an FL client with IFU

$\mathbf{\theta}_i^{n+1}$ is the local update of client $i$ sent to the server after performing $K$ SGD steps on its dataset $D_i$ after initialization with global model $\mathbf{\theta}^n$. Given the contribution $\mathbf{\theta}_i^{n+1} - \mathbf{\theta}^n$ of a client $i$, we define the overall FL increment after aggregations across the set of clients $I$ as
$$\Delta(I, \mathbf{\theta}^n)
:= \frac{1}{|D_I|}\sum_{i \in I} |D_i| \left[\mathbf{\theta}_i^{n+1} - \mathbf{\theta}^n\right] $$

By comparing increments obtained by training on the set of clients $I$, and on the set $I_{-c} := I \setminus \{c\}$ obtained after dropping a given client $c$,  we define the bounded sensitivity after $n$ server aggregations as

$$\Psi(n, c)
:= \sum_{s=0}^{n-1} \Vert \Delta(I, \mathbf{\theta}^s) - \Delta(I_{-c}, \mathbf{\theta}^s)\Vert_2,$$

We show in Theorem 1 that the model sensitivity of FedAvg can be bounded by the bounded sensitivity.

- **Theorem 1.** Under Assumption 1, the model sensitivity of FedAvg when removing a client c after n server aggregations is defined as
$$\alpha(n, c):= \Vert \text{FedAvg}(I, n) - \text{FedAvg}(I_{-c}, n) \Vert_2,$$

### Improving the Tightness of the Sensitivity Bound

Under Assumption \ref{ass:linear_approx}, when considering that clients loss functions are $\mu$-strongly convex and regularized with an L2 norm of weight $\lambda$, we have $\alpha(n , c)\le \Psi(n , c)$ and
$$\Psi(n, c) = \sum_{s=0}^{n-1}\left(1 - \eta (\lambda + \mu )\right)^{\left[(n-1)-s\right]K} \times \Vert \Delta(I, \mathbf{\theta}^s) - \Delta(I_{-c}, \mathbf{\theta}^s) \Vert_2,$$
where $\eta$ and $K$ are respectively the clients' local learning rate and amount of local work.

### Satisfying Definition 1

- **Theorem 2.** Defining
$$\sigma(n, c) := \left[2 \left( \ln(1.25) - \ln(\delta) \right) \right]^{1/2} \epsilon^{-1} \Psi(n , c),$$
the noise perturbation $\sigma(n , c) \mathbf{I}_\mathbf{\theta}$ applied to the global model $\mathbf{\theta}^n$, where $\\mathbf{I}_\mathbf{\theta}$ is the identity matrix, achieves $(\epsilon, \delta)$-unlearnig of client $c$ according to Definition 1.

In what follows, the unlearning procedure will be defined with respect to the sensitivity threshold $\Psi^*$ related to the unlearning budget $(\epsilon, \delta)$ and standard deviation $\sigma$:
$$\Psi^* := \left[2 \left(\ln(1.25) - \ln(\delta)\right)\right]^{-1/2} \epsilon \sigma.$$

### Informed Federated Unlearning (IFU)

To unlearn client $c$, the server identifies the unlearning index $T$ associated to the history of bounded sensitivity metrics, i.e. the most recent global model index such that a perturbation of size $\sigma$ satisfies Theorem \ref{theo:noise_DP}:
$$T := \arg \max_n \left( \Psi(n, c)\le \Psi^* \right)$$

The new global model is obtained after perturbation $\tilde{\mathbf{\theta}} := \mathbf{\theta}^T + \nu$, where $\nu \sim N(0, \sigma^2 \mathbf{I}_\mathbf{\theta})$. Our unlearning criterion is therefore satisfied for $\tilde{\mathbf{\theta}}$ (Theorem 2), and the server can perform $\tilde{N}$ new optimization rounds with FedAvg initialized on $\tilde{\mathbf{\theta}}$. Thanks to the contribution of the remaining clients in $\tilde{\mathbf{\theta}}$, we expect the retraining with IFU to be generally faster than retraining with a random initial model.

Since $\Psi(n, i)$ is strictly increasing with $n$, the server can stop from computing the bounded sensitivity (\ref{eq:def_Psi}) for client $i$ whenever $\Psi(n_i, i ) > \Psi^*$ is verified after $n_i$ optimization rounds. At this point, the model $\mathbf{\theta}^{n_i-1}$ will be selected for the unlearning request of client $i$, as the models at subsequent iterations do not comply with the desired unlearning budget $\Psi^*$. 

##  Sequential FU with SIFU

![1](https://drive.google.com/uc?export=view&id=1wFpr7mkYbWOhbBaF-Zc9Im_SF6Uo_ktn)

We extend the bounded sensitivity with $\Psi_r(n, i)$ to compute the metric of client $i$ at unlearning index $r$ with 
$$\Psi_r(n, i) := \sum_{s=0}^{n-1} \Vert \Delta(I_r, \mathbf{\theta}_r^s) - \Delta(I_r\setminus\{i\}, \mathbf{\theta}_r^s) \Vert_2. $$

When unlearning client $c$ at $r=1$, the metric at $r=0$ is equivalent to the  previous definition of $\Psi$. Also, when computing the metric on a client already unlearned, i.e. $i \notin I_r$, we retrieve $\Psi_r(n, i) = 0$. Finally, for a set of clients $S$, we generalize the bounded sensitivity to
$$\Psi_r(n, S) = \max_{i \in S}\Psi_r(n, i)$$

With SIFU, the selection of the unlearning index $T$ for a request $r$ depends of the past history of unlearning requests. To keep track of the unlearning history, we introduce the oracle $O(r)$ which returns at each request $r$ the coordinates of the history of global models where unlearning has been applied. These coordinates represent the nodes of the training history across unlearning requests. We start with the original sequence of global models obtained at each FL round, i.e. ($\mathbf{\theta}_0^0, \ldots, \mathbf{\theta}_0^{N_0})$. Similarly to IFU, the first unlearning request requires to identify the unlearning index $T_1$ for which the corresponding global model $\mathbf{\theta}_0^{T_1}$ must be perturbed to obtain $\mathbf{\theta}_1^0$ and retrained until convergence, i.e. up to $\mathbf{\theta}_1^{N_1}$. The oracle is updated with the coordinates of the branching $O(1) = \{(0, T_1)\}$, and the current training history is now $(\mathbf{\theta}_0^0, \ldots, \mathbf{\theta}_0^{T_1}, \mathbf{\theta}_1^0, \ldots, \mathbf{\theta}_1^{N_1})$. At the next unlearning request, the server needs to identify the coordinates $(\zeta_r, T_r)$ in the new training history for which unlearning must be applied on the model $\mathbf{\theta}_{\zeta_r}^{T_r}$ to obtain $\mathbf{\theta}_r^0 = \mathbf{\theta}_{\zeta_r}^{T_r} + \mathcal{N}(0, \sigma^2\mathbf{I}_\mathbf{\theta})$. The oracle is subsequently updated with the new set of nodes describing the new branching in the training history. By construction, we have $\zeta_r \le r-1$ and $T_r \le N_{\zeta_r}$. 

More precisely, we define the index $\zeta_r$ associated to the first coordinate in $O(r-1)$ for which the bounded sensitivity (\ref{eq:def_Psi_extended}) of clients in $W_r$ exceeds $\Psi^*$. Formally, we have
$$\zeta_r := \mathbf{I}n_s \{s : \Psi_s(n, W_r) > \Psi^* \text{ and } (s, n) \in O(r-1), r-1 \}.$$

The definition of $T_r$ follows directly from the one of $\zeta_r$. Similarly as for IFU, the unlearning index $T_r$ quantifies the maximum amount of server aggregations starting from the unlearning request index $\zeta_r$ such that the bounded sensitivity $\Psi_{\zeta_r}(n, W_r)$ on this global model is inferior to $\Psi^*$, i.e.
$$T_r := \arg \max_n \{\Psi_{\zeta_r}(n, W_r) \le \Psi^* \}.$$

Finally, we update the oracle $O(r-1)$ to $O(r)$ with the following recurrent equation
$$O(r) = \{(s, n) \in O(r-1) \text{ s.t. } s < \zeta_r, (\zeta_r, T_r) \}.$$

Theorem 3 shows that for a model trained with SIFU after a given training request $r$, $(\epsilon, \delta)$-unlearning is guaranteed for every client belonging to the sets $W_s$, $s\leq r$.

- **Theorem 3.** The model $\mathbf{\theta}_r^{N_r}$ obtained with SIFU satisfies $(\epsilon, \delta)$-unlearning for every client in current and previous unlearning requests, i.e. clients in $\cup_{s=1}^r W_s$.

## Experiments

![2](https://drive.google.com/uc?export=view&id=1M-iIE4kp2YIRMqI_a8prdKpqPG-okcIm)

![3](https://drive.google.com/uc?export=view&id=1ZREpAB529c7Cfz4qHbdhsMc_lqmluqYG)

# References
- Y. Fraboni, R. Vidal, L. Kameni, and M. Lorenzi, Sequential Informed Federated Unlearning: Efficient and Provable Client Unlearning in Federated Optimization. arXiv, 2022. doi: 10.48550/ARXIV.2211.11656. [[Paper](https://arxiv.org/abs/2211.11656)]